In [1]:
#Python-Reddit API Wrapper
import praw 
from psaw import PushshiftAPI

#for connecting to databases
import psycopg2
from sqlalchemy import create_engine


import pandas as pd
import datetime as dt
import numpy as np
import json #for parsing the return from the Google API
import urllib #for passing info to the Google API

import accesses #local file with API certs and database passwords.

In [10]:
nodes = pd.read_csv('data/node_locations.csv')
nodes.head()

,iata_code,lat,lon
0,DAL,32.848103,-96.851206
1,DCA,38.851242,-77.040231
2,DEN,39.856096,-104.673738
3,DSM,41.534133,-93.658796
4,ECP,30.354854,-85.799513


In [24]:
nodes['iata_code'].iloc[:3]

0    DAL
1    DCA
2    DEN
Name: iata_code, dtype: object

In [2]:
api_key = accesses.google_api
url = 'https://maps.googleapis.com/maps/api/geocode/json?'

In [45]:
geo_list = []
for i in nodes['iata_code']:
    geo_dict = {}
    address = i+'+Airport'
    geo_dict['iata_code'] = i
    url_address_api = '{}address={}&key={}'.format(url, address, api_key)
    try:
        with urllib.request.urlopen(url_address_api) as response: 
            js = json.loads(response.read())
        geo_dict['lat'] = js['results'][0]['geometry']['location']['lat']
        geo_dict['lon'] = js['results'][0]['geometry']['location']['lng']
        for component in js['results'][0]['address_components']:
            geo_dict[component['types'][0]] = component['short_name']
        geo_list.append(geo_dict)

    except:
        print('Error in Geocoding.', address, ' not found.')

Error in Geocoding. RBNAirport  not found.
Error in Geocoding. ANCAirport  not found.
Error in Geocoding. BEDAirport  not found.
Error in Geocoding. GAIAirport  not found.
Error in Geocoding. MA5Airport  not found.
Error in Geocoding. BYWAirport  not found.
Error in Geocoding. KEHAirport  not found.
Error in Geocoding. QMAAirport  not found.
Error in Geocoding. CYMAirport  not found.
Error in Geocoding. LAFAirport  not found.
Error in Geocoding. PIRAirport  not found.
Error in Geocoding. PA5Airport  not found.
Error in Geocoding. VT1Airport  not found.
Error in Geocoding. WI5Airport  not found.
Error in Geocoding. CO1Airport  not found.
Error in Geocoding. INTAirport  not found.
Error in Geocoding. SOVAirport  not found.
Error in Geocoding. ARCAirport  not found.
Error in Geocoding. CEMAirport  not found.
Error in Geocoding. RMPAirport  not found.
Error in Geocoding. TALAirport  not found.
Error in Geocoding. POBAirport  not found.
Error in Geocoding. CONAirport  not found.
Error in Ge

In [50]:
df = pd.DataFrame(geo_list)

In [53]:
df.head()

,iata_code,lat,lon,airport,street_number,route,locality,administrative_area_level_2,administrative_area_level_1,country,...,neighborhood,political,administrative_area_level_3,postal_code_suffix,establishment,subpremise,premise,postal_town,bus_station,administrative_area_level_4
0,DAL,32.848103,-96.851206,Dallas Love Field Airport,8008,Herb Kelleher Way,Dallas,Dallas County,TX,US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DCA,38.851242,-77.040231,Ronald Reagan Washington National Airport,NaN,NaN,Arlington,Arlington County,VA,US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DEN,39.856096,-104.673738,Denver International Airport,8500,Peña Blvd,Denver,Denver County,CO,US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DSM,41.534133,-93.658796,Des Moines International Airport,5800,Fleur Dr,Des Moines,Polk County,IA,US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ECP,30.354854,-85.799513,Northwest Florida Beaches International Airport,6300,W Bay Pkwy,Panama City,Bay County,FL,US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
df.to_csv('full_airport.csv')


In [76]:
print('All countries: ', len(df))
df_cleaned = df[df['country']=='US']
print('Just US: ', len(df_cleaned))
df_cleaned = df_cleaned[~df_cleaned['administrative_area_level_1'].isin(['AK', 'HI'])]
print('Just continental US: ', len(df_cleaned))

All countries:  1169
Just US:  1062
Just continental US:  880


In [79]:
df_cleaned.iloc[:, :3].reset_index(drop=True).to_csv('cont_us_nodes.csv', index=False)

In [ ]:
df_final = df_cleaned["origin_iata", "origin_lon", "origin_lat", "dest_iata","dest_lon", "dest_lat"]